# Predicting Industrial Machine Downtime

## 📖 Background
You work for a manufacturer of high-precision metal components used in aerospace, automotives, and medical device applications. Your company operates three different machines on its shop floor that produce different sized components, so minimizing the downtime of these machines is vital for meeting production deadlines.

Your team wants to use a data-driven approach to predicting machine downtime, so proactive maintenance can be planned rather than being reactive to machine failure. To support this, your company has been collecting operational data for over a year and whether each machine was down at those times.

In this first level, you're going to explore and describe the data. This level is aimed towards beginners. If you want to challenge yourself a bit more, check out level two!

## 💾 The data

The company has stored the machine operating data in a single table, available in `'data/machine_downtime.csv'`.

#### Each row in the table represents the operational data for a single machine on a given day:
- `"Date"` - the date the reading was taken on.
- `"Machine_ID"` - the unique identifier of the machine being read.
- `"Assembly_Line_No"` - the unique identifier of the assembly line the machine is located on.
- `"Hydraulic_Pressure(bar)"`, `"Coolant_Pressure(bar)"`, and `"Air_System_Pressure(bar)"` - pressure measurements at different points in the machine.
- `"Coolant_Temperature"`, `"Hydraulic_Oil_Temperature"`, and `"Spindle_Bearing_Temperature"` - temperature measurements (in Celsius) at different points in the machine.
- `"Spindle_Vibration"`, `"Tool_Vibration"`, and `"Spindle_Speed(RPM)"` - vibration (measured in micrometers) and rotational speed measurements for the spindle and tool.
- `"Voltage(volts)"` - the voltage supplied to the machine.
- `"Torque(Nm)"` - the torque being generated by the machine.
- `"Cutting(KN)"` - the cutting force of the tool.
- `"Downtime"` - an indicator of whether the machine was down or not on the given day.

In [ ]:
# import the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.impute import KNNImputer
import math
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

## Data Analysis

In [ ]:
# read the data and print the first few rows
machine_downtime = pd.read_csv('C://Users/Administrator/Documents/Data Science Projects/Machine-Downtime-Prediction/data/machine_downtime.csv')
machine_downtime.head()

#### Useful Functions

In [28]:
def plot_numeric_features(df, plot_type="boxplot", cols=4, figsize_multiplier=4):
    """
    Plots numerical columns from a dataframe in a grid layout.

    Parameters:
    df (pd.DataFrame): The dataframe containing numerical columns.
    plot_type (str): Type of plot to generate ("boxplot", "histogram", or "violin").
    cols (int): Number of columns in the grid.
    figsize_multiplier (int): Multiplier to adjust figure size dynamically.

    Returns:
    None: Displays the generated plots.
    """
    # Select only numerical columns
    feature_numeric = df.select_dtypes(include=['number'])

    # Determine grid size
    num_cols = feature_numeric.shape[1]
    rows = math.ceil(num_cols / cols)  # Compute required rows dynamically

    # Create subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, figsize_multiplier * rows))
    axes = axes.flatten()  # Flatten to make iteration easier

    # Iterate over each numerical column and plot
    for i, col in enumerate(feature_numeric.columns):
        if plot_type == "boxplot":
            sns.boxplot(y=feature_numeric[col], ax=axes[i])
        elif plot_type == "histogram":
            sns.histplot(feature_numeric[col], ax=axes[i], kde=True, bins=30)
        elif plot_type == "violin":
            sns.violinplot(y=feature_numeric[col], ax=axes[i])
        else:
            raise ValueError("Invalid plot type. Choose 'boxplot', 'histogram', or 'violin'.")

        axes[i].set_title(col)

    # Remove any empty subplots
    for i in range(num_cols, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

## Data Wrangling

In [ ]:
# create a copy of the original data
machine_downtime_ori = machine_downtime.copy()

### 1. inconsistent data types and NaN values 
+ The `Date` column is type **object** instead of type **datetime**, We will also convert the `Assembly_Line_No` and `Downtime` from **Object** to **Category** typr
+ There are NaN values in majority of the columns. we need to do further analysis on that
+ The other column data types are okay


In [ ]:
# get data types of columns
machine_downtime.info()

Looking at the plot below, there is no pattern to the missing values as they seem to be generally random.

### 2. Value range constraint 
+ There are negative values in `Hydraulic_pressure` and `Spindle_Vibration` column. we will investigate that
+ We would convert the `cutting(kN)` SI unit column to **newton(N)** from **KiloNewton(kN)** and the `Spindle_Speed` from **Rotation per minute (RPM)** to **Rotation per seconds(RPS)** since all other variable are in their standard units
+ for the `Date` column, we will drill into it more during the data correction phase
+ `Coolant temperature` of 4C seems unusual. we will investigate later and also a pretty high teprature of 98C seems unusual too
+ The `tool vibration` column also has a minimum value of 2.16 which when when compared with others seems impossible to be.
+ we aso have a minimum value for `spindle speed` and `Torque` to be zero. This is possible only of the machine is not operating at that moment or a fault in the sensor





In [ ]:
# get the summray statistics of the data
machine_downtime_ori.describe()

#### 2.1 Detecting Outliers in The Data
From the boxplots in the image, here are some key inferences about the numerical data:

1. Presence of Outliers
> + Several columns show a significant number of outliers, which are represented as dots outside the whiskers.
> + High number of outliers:
Voltage(volts), Torque(Nm), Air_System_Pressure(Pa), Tool_Vibration, and Coolant_Pressure(Pa)
These variables have extreme values that could indicate measurement anomalies, rare events, or genuine variations in the data.
2. Distribution Insights
> + Most of the boxplots show a relatively symmetric distribution, but some have skewness:
>  + Positively Skewed (Right-tailed): Hydraulic_Oil_Temperature, Voltage(volts), Cutting(N)
Negatively Skewed (Left-tailed): Spindle_Speed(RPS) (due to a low extreme outlier)
3. Spread of Data (Variability)
> + Columns with High Spread:
Hydraulic_Pressure(Pa), Voltage(volts), and Cutting(N) have a large range of values.
This could suggest that these parameters vary significantly in the dataset.
> + Columns with Low Spread:
Spindle_Vibration and Spindle_Bearing_Temperature have a smaller range, meaning the values are more consistent.
4. Potential Data Quality Issues
> + Some extreme outliers might be due to sensor errors, faulty readings, or noise in the dataset.
> + Others might be a normal range of values for the specific feature in question

If these outliers are not expected in the domain (e.g., industrial sensors in automation and robotics), it might be worth investigating their source.

In our data cleaning phase, only the extreme maximum amd extreme minimum will be removed from the record and this will be for specific columns like `Coolant_Pressure`, `Coolant_temperature`, `Spindle_Speed`, `Torque`.

In [ ]:
# Select only numerical columns
feature_numeric = machine_downtime_ori.select_dtypes(include=['number'])

# Define grid size (e.g., 3 rows, 4 columns)
num_cols = feature_numeric.shape[1]
cols = 4  # Number of columns in the grid
rows = math.ceil(num_cols / cols)  # Compute number of rows dynamically

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(15, 4 * rows))

# Flatten axes array if needed (for easier iteration)
axes = axes.flatten()

# Plot each numerical column in a separate subplot
for i, col in enumerate(feature_numeric.columns):
    sns.boxplot(y=feature_numeric[col], ax=axes[i])
    axes[i].set_title(col)

# Remove any empty subplots
for i in range(len(feature_numeric.columns), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### 3. Uniqueness Constraints
+ No spelling irregularities in the `Downtime` column
+ There are no duplicates present


In [ ]:
# check the values of `downtime` column
machine_downtime_ori.Downtime.unique()

In [ ]:
# check for duplicates
duplicate_records = machine_downtime_ori.duplicated().sum()
print(f"the number of duplicate is {duplicate_records}")

## Data Cleaning
We will be cleaning our data after performing some initial exploration. 

### 1.1 Change `Date` column to `datatime` type

+ We will convert the Date column data type from object type to datetime
+ There seems to be disparity in the arrangement of the records as they are not arranged in the right order. I will sort the dataframe by date column in ascending order. 


In [ ]:
# change the date column to datetime
machine_downtime_ori['Date'] = pd.to_datetime(machine_downtime_ori['Date'],
                                              errors='coerce')
# verify 
machine_downtime_ori.Date.dtype

Looking at the first few rows, the row with the min date value does not correspond to the first row and the same can be seen for the last row as it does not correspond the record with the max date value. we will address this

In [ ]:
machine_downtime_ori.head()

In [ ]:
machine_downtime_ori.tail()

In [ ]:
# get the max date value
machine_downtime_ori.Date.max()

In [ ]:
# get the min date value
machine_downtime_ori.Date.min()

In [ ]:
# rearrange the dataframe based on the date column
machine_downtime_ori = machine_downtime_ori.sort_values(by = 'Date', ignore_index= True)
machine_downtime_ori.head()     # print the first few row to verify this

We also have to convert the `Machine_ID`, `Downtime` and `Assembly_Line_No` from Object data type to categorical

In [ ]:
# connvert the columns of object type to Category type
object_column = machine_downtime_ori.select_dtypes(include='object').columns
machine_downtime_ori[object_column] = machine_downtime_ori[object_column].astype('category')

# verufy 
machine_downtime_ori[object_column].dtypes

### 1.2 Imputing Missing Values

We will drill down on the NaN values to determine whether to impute or drop the NaN values
 

Looking at the diagram below, which presents the missing values in a graphical format using the missingno library, I observed that there is no clear pattern to the missing values, as they appear randomly. However, I noticed a particular section in the data where the `torque` and `voltage` columns contain NaN values. This could be due to sensor failure, system downtime, or specific operating conditions where no readings were recorded. Similarly, the `coolant pressure` and `column temperature` columns exhibit the same behavior, suggesting potential sensor-related issues or machine inactivity during those periods.

In [ ]:
# plot the missing value matrix 
msno.matrix(machine_downtime_ori)
plt.show()

Let us try to plot the distribution of the numerical column to see the underlying patter in order to use the appropriate imputation method
from the distibution plots below, we can see that some features exhibit normal distribution while others were either right-skewed or left-skewed.We will be using an advanced method of imputation - The KNN imputation to fill in our missing values in each column

In [ ]:
# create a list for all the numerical columns with NaN
numerical_cols_with_nan = machine_downtime_ori.select_dtypes(include=np.number)\
                          .columns[machine_downtime_ori.select_dtypes(include=np.number).isnull().any()].tolist()

for col in numerical_cols_with_nan:
    plt.figure()  # Create a new figure for each plot
    machine_downtime_ori[col].hist() # or df[col].plot.hist()
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()
  

KNN imputation will be used to address missing values across the dataset, leveraging inter-variable relationships to estimate missing data points and ensure a complete dataset for analysis. KNN imputation estimates missing values by considering the values of similar data points, thereby ensuring a complete and consistent dataset for analysis.

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3) # initialize the imputer
machine_downtime_ori[numerical_cols_with_nan] = imputer.fit_transform(machine_downtime_ori[numerical_cols_with_nan]) # Impute based on both columns

# verify that there is no more missing values
machine_downtime_ori.isna().sum()

#### 1.3 Convert Columns values to its S.I unit values

+ We will convert the `Cutting (kN)` column from kN to Newton
+ we will also round off the values of each numerical to have conistency in the decimal place.
+ WE will convert the `Spindle Speed` from RPM to RPS 
+ convert the pressure columns from bar to Pa




In [ ]:
import re  # Import the regular expression module

def convert_units(df):
    """Converts pressure from bar to Pa, force from kN to N, and spindle speed from RPM to RPS,
       rounding the results to 1 decimal place. Handles multiple pressure columns 
       containing "pressure" and "(bar)".

    Args:
        df: The Pandas DataFrame containing the data.

    Returns:
        A new Pandas DataFrame with converted units, rounded to 1 decimal place.
        Returns the original DataFrame if no conversion is needed.
    """

    # Pressure (bar to Pa) - Handles multiple pressure columns containing "Pressure" and "(bar)"
    for col in df.columns:
        if "Pressure" in col and "(bar)" in col:
            new_col_name = re.sub(r"\(bar\)", "(Pa)", col) # Use regex to replace (bar) with (Pa)
            df[new_col_name] = (df[col].astype(float) * 100000).round(1) #convert to float before multiplication
            df = df.drop(columns=[col])

    # Force (kN to N)
    if 'Cutting(kN)' in df.columns:
        df['Cutting(N)'] = (df['Cutting(kN)'] * 1000).round(1)
        df = df.drop(columns=['Cutting(kN)'])

    # Spindle Speed (RPM to RPS)
    if 'Spindle_Speed(RPM)' in df.columns:
        df['Spindle_Speed(RPS)'] = (df['Spindle_Speed(RPM)'] / 60).round(1)
        df= df.drop(columns=['Spindle_Speed(RPM)'])

    return df

# apply the function to the dataframe
machine_downtime_ori = convert_units(machine_downtime_ori)
machine_downtime_ori.head()

#### 1.4 Negative values in Hydraulic pressure column and Spindle vibration column
During my analysis, I encountered negative values in my sensor data, but they appear in different rows for hydraulic pressure and spindle vibration. I have one row with a negative hydraulic pressure reading. Negative hydraulic pressure in our system is highly improbable, suggesting a sensor or data acquisition error.

Separately, I have another row with negative spindle vibration value.  These negative values, while valid in the context of vibration, is quite small (less than 0.5). Unlike the pressure readings, negative values for spindle vibration are perfectly normal.  Vibration is an oscillatory motion, meaning the spindle moves back and forth.  A negative vibration value simply indicates the direction of movement – in this case, towards the sensor

Given that it's only one row for each feature, I'm inclined to believe it's an isolated incident and not representative of the overall system behavior.  I've decided to remove this single row for each column from my dataset.  Since it's just one data point, removing it shouldn't significantly impact my overall analysis, and it will prevent this erroneous data from skewing any calculations or models I might build


In [ ]:
# check the number of rows where the hydraulic pressure is negative
hydraulic_neg = machine_downtime_ori[machine_downtime_ori['Hydraulic_Pressure(Pa)'] <= 0]
hydraulic_neg

In [ ]:
# check the number of rows where the Spindle Vibration is negative
spindle_neg = machine_downtime_ori[machine_downtime_ori['Spindle_Vibration'] <= 0]
spindle_neg

In [ ]:
# drop the rows with negative value
machine_downtime_ori = machine_downtime_ori.drop(machine_downtime_ori[(machine_downtime_ori['Hydraulic_Pressure(Pa)'] < 0)\
                       | ((machine_downtime_ori['Spindle_Vibration'] < 0))].index)

# check for verification
machine_downtime_ori[(machine_downtime_ori['Hydraulic_Pressure(Pa)'] < 0)\
                       | ((machine_downtime_ori['Spindle_Vibration'] < 0))]

#### Removing Extreme Outliers
Since most of the columns follow a normal distribution, but some are skewed, you can apply different outlier detection methods per column type:

+ we will Use Z-score (±3σ Rule) for Normally Distributed Columns
+ we use Use IQR (3 × IQR Rule) for Skewed Columns

In our data cleaning phase, only the extreme maximum amd extreme minimum will be removed from the record and this will be for specific columns like `Coolant_Pressure(Pa)`, `Coolant_temperature`, `Spindle_Speed(RPS)`, `Torque(Nm)`.

1. Identify Normal vs. Skewed Columns
> + First, determine which columns are normally distributed and which are skewed:

2. Remove Outliers by Method
> + Now, apply Z-score for normal columns and IQR for skewed columns:

In [ ]:
# Function to classify columns
def classify_columns(df):
    
    """
    Classifies numerical columns into normal or skewed distributions based on the Shapiro-Wilk test.
    
    Parameters:
    df (pd.DataFrame): The input dataframe containing numerical data.
    
    Returns:
    tuple: A tuple containing:
        - normal_cols (list): List of column names that follow a normal distribution.
        - skewed_cols (list): List of column names that are skewed.
    """
    normal_cols = []
    skewed_cols = []
    
    for col in df.select_dtypes(include=['number']).columns:
        stat, p_value = stats.shapiro(df[col].dropna())  # Shapiro-Wilk test for normality
        if p_value > 0.05:  # p-value > 0.05 means normal distribution
            normal_cols.append(col)
        else:
            skewed_cols.append(col)
    
    return normal_cols, skewed_cols

# Get normal and skewed columns
normal_cols, skewed_cols = classify_columns(machine_downtime_ori)
print("Normal Distribution:", normal_cols)
print("Skewed Distribution:", skewed_cols)

In [ ]:
def remove_outliers(df, columns_to_clean):
    """
    Removes extreme outliers only for specific numeric columns using:
    - Z-score (±3 standard deviations) for normally distributed columns.
    - IQR (Interquartile Range with 3 × IQR Rule) for skewed columns.
    
    Parameters:
    df (pd.DataFrame): The input dataframe.
    columns_to_clean (list): List of specific column names where outlier removal should be applied.
    
    Returns:
    pd.DataFrame: A cleaned dataframe with extreme outliers removed only from the specified columns.
    """

    for col in columns_to_clean:
        # Check if column exists
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in dataframe.")
            continue
        
        # Detect normality of the column
        stat, p_value = stats.shapiro(df[col].dropna())  
        is_normal = p_value > 0.05  # If p > 0.05, assume normal distribution

        if is_normal:
            # Apply Z-score for normal distributions (±3σ Rule)
            z_scores = np.abs(stats.zscore(df[col]))
            df = df[z_scores < 3]  # Keep values within ±3σ
        else:
            # Apply IQR for skewed distributions (3 × IQR Rule)
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    df.reset_index(drop=True, inplace=True)
    return df


In [ ]:
# get the list of the classified features
normal_cols, skewed_cols = classify_columns(machine_downtime_ori)  # Identify distributions

# specify columns to remove outliers from
columns_remove_outliers = ['Coolant_Pressure(Pa)', 'Coolant_Temperature', 'Spindle_Speed(RPS)', 'Torque(Nm)']
machine_downtime_ori = remove_outliers(machine_downtime_ori, columns_remove_outliers)  # Remove extreme outliers

print(f"New dataset size after removing extreme outliers: {machine_downtime_ori.shape}")

#### Save The cleaned Data

In [ ]:
# saved the cleaned data to a csv file
machine_downtime_ori.to_csv('C://Users/Administrator/Documents/Data Science Projects/Machine-Downtime-Prediction/data/machine_downtime_cleaned.csv', 
                            index=False)